In [2]:
!pip install gym

     ---------------------------------------- 0.0/721.7 kB ? eta -:--:--
     -------------------------------------- 721.7/721.7 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827632 sha256=cdb0c524c30af67fb08b16ad16eccd05333fa64eace8b7c7648d2286ed075751
  Stored in directory: c:\users\ashok adithya\appdata\local\pip\cache\wheels\17\79\65\7afedc162d858b02708a3b8f7a6dd5b1000dcd5b0f894f7cc1
Successfully built gym


In [8]:
import numpy as np
import gym
from gym import spaces

class DigitalTwin5GEnv(gym.Env):
    def __init__(self):
        super(DigitalTwin5GEnv, self).__init__()
        self.observation_space = spaces.Box(low=0, high=1, shape=(60, 11), dtype=np.float32)
        self.action_space = spaces.Discrete(3)  # emBB, URLLC, mMTC
        self.current_step = 0
        self.state = np.random.rand(60, 11)

    def reset(self):
        self.current_step = 0
        self.state = np.random.rand(60, 11)
        return self.state

    def step(self, action):
        reward = np.random.rand()  # Replace with real metric logic
        done = self.current_step >= 200
        self.current_step += 1
        self.state = np.random.rand(60, 11)  # Simulate next input
        return self.state, reward, done, {}


In [9]:
import numpy as np
from tensorflow.keras.models import load_model
from env.network_env import DigitalTwin5GEnv
from agent.dqn_agent import DQNAgent

demand_model = load_model("demandPredictionTanh2.h5")
classification_model = load_model("ann_model.h5")

env = DigitalTwin5GEnv()
agent = DQNAgent(env)

episodes = 20

for ep in range(episodes):
    state = env.reset()
    total_reward = 0
    done = False

    while not done:
        demand = demand_model.predict(state.reshape(1, 60, 11), verbose=0).flatten()[0]
        traffic_type = np.argmax(classification_model.predict(state.reshape(1, 60, 11), verbose=0), axis=1)[0]

        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

    print(f"✅ Episode {ep+1}: Total Reward = {total_reward:.2f}")
    agent.replay()


ModuleNotFoundError: No module named 'env'